[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jkitchin/s26-06642/blob/main/dsmles/13-model-interpretability/model-interpretability.ipynb)

In [ ]:
! pip install -q pycse
from pycse.colab import pdf

# Module 11: Model Interpretability

Understanding why models make their predictions.

## Learning Objectives

1. Understand the importance of interpretability
2. Use feature importance from tree-based models
3. Apply SHAP values for model explanation
4. Create partial dependence plots
5. Interpret individual predictions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import PartialDependenceDisplay
import xgboost as xgb
import shap

## Why Interpretability Matters: The Trust Problem

Machine learning models are increasingly powerful, but power without understanding is dangerous. In chemical engineering, we can't just accept predictions—we need to understand *why*.

### The Interpretability Imperative

| Scenario | Why You Need Interpretability |
|----------|------------------------------|
| Safety-critical decisions | Regulators require explanations |
| Process optimization | Need to know which variables to adjust |
| Model debugging | Unexpected predictions need diagnosis |
| Knowledge discovery | ML can reveal unknown relationships |
| Stakeholder trust | Operators won't use black boxes |

### The Accuracy-Interpretability Tradeoff

Historically, there was a perceived tradeoff:

| Model Type | Interpretability | Accuracy |
|------------|-----------------|----------|
| Linear regression | High | Limited |
| Decision trees | High | Moderate |
| Random forests | Low | High |
| Neural networks | Very low | Very high |

**Modern interpretability tools (SHAP, LIME) break this tradeoff.** We can now explain complex models without sacrificing accuracy.

### Two Types of Explanation

1. **Global**: What features matter overall?
2. **Local**: Why did the model make *this specific* prediction?

Both are important. Global explanations validate the model. Local explanations enable debugging and trust.

In [ ]:
# Create a chemical process dataset
np.random.seed(42)
n_samples = 500

# Process variables
temperature = np.random.uniform(300, 500, n_samples)
pressure = np.random.uniform(1, 20, n_samples)
catalyst_loading = np.random.uniform(0.5, 5, n_samples)
residence_time = np.random.uniform(5, 60, n_samples)
feed_purity = np.random.uniform(0.9, 1.0, n_samples)
stirrer_speed = np.random.uniform(100, 500, n_samples)

# Target: Product yield (complex nonlinear relationship)
yield_product = (
    40 * (1 - np.exp(-0.01 * (temperature - 300))) *  # Temperature activation
    (1 - np.exp(-0.1 * pressure)) *                    # Pressure effect
    np.tanh(0.5 * catalyst_loading) *                  # Catalyst saturation
    (1 - np.exp(-0.05 * residence_time)) *             # Time effect
    feed_purity *                                       # Linear purity effect
    (1 + 0.001 * (stirrer_speed - 300))                # Minor mixing effect
)
yield_product = yield_product + np.random.normal(0, 2, n_samples)
yield_product = np.clip(yield_product, 0, 100)

# Create DataFrame
feature_names = ['temperature', 'pressure', 'catalyst_loading', 
                 'residence_time', 'feed_purity', 'stirrer_speed']
df = pd.DataFrame({
    'temperature': temperature,
    'pressure': pressure,
    'catalyst_loading': catalyst_loading,
    'residence_time': residence_time,
    'feed_purity': feed_purity,
    'stirrer_speed': stirrer_speed,
    'yield': yield_product
})

X = df[feature_names].values
y = df['yield'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dataset shape: {df.shape}")
df.describe()

In [ ]:
# Train models
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)

print(f"Random Forest Test R²: {rf.score(X_test, y_test):.4f}")
print(f"XGBoost Test R²: {xgb_model.score(X_test, y_test):.4f}")

## Feature Importance: The First Step

Tree-based models (Random Forest, XGBoost) provide built-in feature importance. This is often your first look at what the model learned.

### How It's Calculated

| Method | Measure | Pros | Cons |
|--------|---------|------|------|
| **Impurity-based** (RF) | Total reduction in impurity | Fast, built-in | Biased toward high-cardinality features |
| **Gain-based** (XGBoost) | Average gain when feature is used | Fast, built-in | Can vary between runs |
| **Permutation** | Accuracy drop when feature is shuffled | Model-agnostic, unbiased | Slower, affected by correlations |

### The Limitation

Feature importance tells you **what** matters, but not **how** it matters. Is temperature positively or negatively correlated with yield? Does the effect saturate? These questions require deeper analysis.

In [ ]:
# Feature importance from Random Forest
importance_rf = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Random Forest
axes[0].barh(importance_rf['Feature'], importance_rf['Importance'], 
             edgecolor='black', color='steelblue')
axes[0].set_xlabel('Feature Importance')
axes[0].set_title('Random Forest Feature Importance')
axes[0].grid(True, alpha=0.3, axis='x')

# XGBoost
importance_xgb = pd.DataFrame({
    'Feature': feature_names,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=True)

axes[1].barh(importance_xgb['Feature'], importance_xgb['Importance'], 
             edgecolor='black', color='coral')
axes[1].set_xlabel('Feature Importance')
axes[1].set_title('XGBoost Feature Importance')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## SHAP Values: The Gold Standard for Interpretability

SHAP (SHapley Additive exPlanations) is based on game theory. It answers: **"How much did each feature contribute to this prediction?"**

### The Shapley Value Concept

Imagine features as players in a game (predicting the target). The Shapley value assigns credit fairly by considering all possible combinations of features and averaging each feature's marginal contribution.

### Why SHAP Is Special

1. **Consistency**: If a feature's contribution increases, its SHAP value won't decrease
2. **Local accuracy**: SHAP values sum to the difference between prediction and baseline
3. **Theoretical foundation**: Based on 60+ years of game theory research
4. **Signs and magnitudes**: Positive SHAP = pushed prediction up; magnitude = how much

### Interpreting SHAP Plots

| Plot Type | What It Shows | Use For |
|-----------|---------------|---------|
| Summary | Feature importance + direction + distribution | Global overview |
| Waterfall | Single prediction breakdown | Explaining individual decisions |
| Dependence | Feature effect vs feature value | Understanding relationships |
| Force | Real-time feature contributions | Interactive exploration |

In [ ]:
# Create SHAP explainer
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)

print(f"SHAP values shape: {shap_values.shape}")
print(f"Each row: SHAP value for each feature for one prediction")

In [ ]:
# Summary plot: global feature importance with direction
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, feature_names=feature_names, show=False)
plt.title('SHAP Summary Plot')
plt.tight_layout()
plt.show()

**Reading the SHAP summary plot:**

This is one of the most information-dense visualizations in ML. Here's how to read it:

- **Each row** is a feature, sorted by importance (most important at top)
- **Each point** is one sample from the test set
- **X-axis** shows the SHAP value (contribution to prediction)
- **Color** shows the feature value (red=high, blue=low)

**What we see:**
- **temperature**: Red points (high temp) have positive SHAP values → higher temperature increases predicted yield
- **catalyst_loading**: Same pattern—more catalyst increases yield
- **feed_purity**: Red points on the right → higher purity increases yield
- **pressure**: Positive relationship, but less spread than temperature
- **residence_time**: Smaller effect, but still positive
- **stirrer_speed**: Minimal effect (narrow horizontal spread)

**Physical validation**: All effects are positive (more of each variable increases yield), which matches our synthetic data and general chemical intuition. If we saw negative effects for temperature or catalyst, we'd question the model or our data!

In [ ]:
# Bar plot of mean absolute SHAP values
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, feature_names=feature_names, 
                  plot_type='bar', show=False)
plt.title('Mean |SHAP| Feature Importance')
plt.tight_layout()
plt.show()

## Explaining Individual Predictions

In [ ]:
# Explain a single prediction
sample_idx = 0
sample = X_test[sample_idx]
prediction = xgb_model.predict([sample])[0]
actual = y_test[sample_idx]

print(f"Sample features:")
for name, val in zip(feature_names, sample):
    print(f"  {name}: {val:.2f}")
print(f"\nPredicted yield: {prediction:.2f}")
print(f"Actual yield: {actual:.2f}")

In [ ]:
# Waterfall plot for single prediction
plt.figure(figsize=(10, 6))
shap.plots.waterfall(shap.Explanation(
    values=shap_values[sample_idx],
    base_values=explainer.expected_value,
    data=X_test[sample_idx],
    feature_names=feature_names
), show=False)
plt.title(f'SHAP Waterfall: Sample {sample_idx}')
plt.tight_layout()
plt.show()

In [ ]:
# Force plot for single prediction
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[sample_idx], 
                X_test[sample_idx], feature_names=feature_names)

## SHAP Dependence Plots

How does a feature affect predictions across all samples?

In [ ]:
# Dependence plots for key features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for ax, (idx, name) in zip(axes.flat, enumerate(feature_names)):
    shap.dependence_plot(idx, shap_values, X_test, 
                         feature_names=feature_names, ax=ax, show=False)
    ax.set_title(f'{name}')

plt.tight_layout()
plt.show()

## Partial Dependence Plots: The Marginal Effect

Partial Dependence Plots (PDPs) show how a feature affects predictions **on average**, while marginalizing over other features.

### How It Works

1. Pick a feature (e.g., temperature)
2. Create a grid of values for that feature
3. For each grid value, replace all samples' temperature with that value
4. Average the predictions
5. Plot average prediction vs temperature

### PDPs vs SHAP Dependence

| Aspect | PDP | SHAP Dependence |
|--------|-----|-----------------|
| Shows | Marginal effect | Actual contribution |
| Interactions | Hidden (averaged out) | Visible (colored by interaction) |
| Interpretation | "If we set T=400, yield is ~X" | "For samples where T=400, T contributed ~X" |

### 2D PDPs: Interaction Effects

Two-feature PDPs reveal interactions. If the effect of temperature depends on pressure, the 2D plot will show it—something hidden in 1D plots.

In [ ]:
# Partial dependence plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for ax, idx in zip(axes.flat, range(len(feature_names))):
    PartialDependenceDisplay.from_estimator(
        xgb_model, X_train, [idx], 
        feature_names=feature_names, ax=ax
    )
    ax.set_title(feature_names[idx])

plt.tight_layout()
plt.show()

In [ ]:
# 2D interaction plot: temperature vs pressure
fig, ax = plt.subplots(figsize=(10, 8))

PartialDependenceDisplay.from_estimator(
    xgb_model, X_train, [(0, 1)],  # temperature and pressure
    feature_names=feature_names, ax=ax
)
ax.set_title('Interaction: Temperature × Pressure')
plt.tight_layout()
plt.show()

## Comparing Model Interpretations

In [ ]:
# Compare feature importance methods
# 1. Impurity-based (RF)
# 2. Gain-based (XGBoost)
# 3. SHAP-based

shap_importance = np.abs(shap_values).mean(axis=0)

comparison = pd.DataFrame({
    'Feature': feature_names,
    'RF Impurity': rf.feature_importances_,
    'XGB Gain': xgb_model.feature_importances_,
    'SHAP': shap_importance / shap_importance.sum()  # Normalize
})

# Normalize all for comparison
for col in ['RF Impurity', 'XGB Gain', 'SHAP']:
    comparison[col] = comparison[col] / comparison[col].sum()

print("Feature Importance Comparison (normalized):")
print(comparison.round(3).to_string(index=False))

In [ ]:
# Visualize comparison
comparison_melted = comparison.melt(id_vars='Feature', var_name='Method', value_name='Importance')

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(feature_names))
width = 0.25

ax.bar(x - width, comparison['RF Impurity'], width, label='RF Impurity', edgecolor='black')
ax.bar(x, comparison['XGB Gain'], width, label='XGB Gain', edgecolor='black')
ax.bar(x + width, comparison['SHAP'], width, label='SHAP', edgecolor='black')

ax.set_xticks(x)
ax.set_xticklabels(feature_names, rotation=45, ha='right')
ax.set_ylabel('Normalized Importance')
ax.set_title('Feature Importance: Method Comparison')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Domain Validation

Check if model insights match domain knowledge.

In [ ]:
# Expected relationships (from process knowledge):
# 1. Temperature: Positive effect (Arrhenius)
# 2. Pressure: Positive effect (Le Chatelier)
# 3. Catalyst: Positive effect (saturation curve)
# 4. Residence time: Positive effect (reaction completion)
# 5. Feed purity: Positive effect (less side reactions)
# 6. Stirrer speed: Minor positive effect (mixing)

# Check SHAP dependence trends
print("Domain Validation:")
print("="*50)

for idx, name in enumerate(feature_names):
    correlation = np.corrcoef(X_test[:, idx], shap_values[:, idx])[0, 1]
    direction = "Positive" if correlation > 0 else "Negative"
    print(f"{name}: {direction} effect (r = {correlation:.3f})")

## Summary: The Interpretability Toolkit

### Method Comparison

| Method | Speed | Insight Type | Best For |
|--------|-------|--------------|----------|
| Feature importance | Fast | Global ranking | Quick overview |
| SHAP summary | Moderate | Global + direction | Detailed analysis |
| SHAP waterfall | Fast | Single prediction | Debugging, explanation |
| Dependence plots | Fast | Feature relationships | Understanding effects |
| Partial dependence | Moderate | Marginal effects | Smooth trends |
| 2D PDP | Slow | Interactions | Discovering dependencies |

### The Interpretation Workflow

1. **Start with feature importance**: Quick overview of what matters
2. **Use SHAP summary**: See direction and distribution of effects
3. **Check domain validity**: Do the effects make physical sense?
4. **Debug with waterfall**: Explain unexpected predictions
5. **Explore with dependence plots**: Understand nonlinear relationships

### Key Takeaways

1. **Interpretability enables trust**: Stakeholders need to understand why
2. **SHAP is theoretically grounded**: Use it as your primary tool
3. **Validate against domain knowledge**: Models should make sense
4. **Local + global**: You need both types of explanation
5. **Different methods, different insights**: Use multiple approaches

### Common Pitfalls

- Trusting feature importance without checking direction of effects
- Ignoring interactions (use 2D PDPs and SHAP dependence)
- Not validating against domain knowledge
- Over-interpreting small differences in importance
- Forgetting that correlation ≠ causation (even with SHAP)

## Course Conclusion

You've now learned a complete toolkit for data science in chemical engineering:

| Module | Skills |
|--------|--------|
| NumPy & Pandas | Data manipulation |
| Dimensionality reduction | PCA, t-SNE, UMAP |
| Regression | Linear, regularized, nonlinear |
| Ensemble methods | Random Forest, XGBoost |
| Clustering | K-means, hierarchical, DBSCAN |
| Uncertainty | pycse, Gaussian Processes |
| Interpretability | SHAP, partial dependence |

**Now apply these tools to your own problems!** The best way to learn is by doing. Start with a real dataset from your research or industry, and work through the analysis pipeline we've covered.

---

## The Catalyst Crisis: Chapter 13 - "The Reveal"

*A story about interpretability, communication, and the power of explanation*

---

The boardroom was larger than Alex expected. Twelve executives around a polished table, the ChemCorp logo glowing on the wall screen. Frank Morrison sat near the end, arms crossed, but with something that might have been pride in his expression.

"Thank you for your time," Professor Pipeline began. "The Data Academy team has spent a semester investigating the catalyst crisis. Alex Chen will present the findings."

Alex took a breath and clicked to the first slide.

"Eighteen months ago, ChemCorp's flagship reactor started producing inconsistent batches. Good yield, bad yield, no apparent pattern." She advanced to the t-SNE plot. "We found the pattern."

She walked them through the journey. Missing data that wasn't random. An outlier that wasn't an error. Dimensions collapsing from two hundred to five. Models that predicted well but couldn't explain why.

"The answer came when we stopped asking 'what predicts yield' and started asking 'why do certain batches fail.'" She pulled up the SHAP values—the interpretability layer that explained individual predictions.

"This is a single batch that failed." The waterfall chart showed features pushing the prediction up and down. "Temperature contributed positively—it was in a good range. Pressure, neutral. But catalyst age pushed the prediction strongly negative. This batch used catalyst from Lot 7,392—Cluster 3 material, manufactured in April of last year."

She clicked through more examples. The pattern was consistent. Every failed batch had the same story: catalyst from the March-August window, trace metal ratios slightly off, surface area slightly low. Each property within spec. The combination, deadly.

"We traced it back to your supplier." Final slide. "They changed their calcination process in February. The change was undocumented and unannounced. The new process produces catalyst that meets all specifications individually but performs poorly in your reactor."

The VP of Operations spoke first. "You're certain?"

"We presented this analysis to the supplier yesterday. They confirmed the process change." Alex allowed herself a small smile. "They're implementing corrective action."

Silence around the table. Then, slowly, nods. The skepticism she'd sensed when she walked in was dissolving.

Frank Morrison spoke up. "I've been running reactors for thirty years. I know this plant better than anyone." He paused. "I couldn't have found this. Not without the data analysis. Not without the modeling." He looked at Alex. "I was wrong to be skeptical."

"You were right to be skeptical," Alex said. "Until we could explain why the model was right, skepticism was reasonable. Predictions without explanations are just black boxes. You needed to understand."

As the meeting wrapped up, Professor Pipeline caught Alex's eye. The smallest of nods.

Later, at the Data Academy mystery board, Alex removed the sticky notes one by one. Eighteen clues, accumulated over a semester, now resolved into a single answer.

She saved the last note for her notebook: **The mystery wasn't in the data. It was in what we assumed we already knew.**

---

## Epilogue

Three weeks after the presentation, Alex sat in the graduate student lounge, coffee in hand. The semester was over. The mystery was solved. And yet...

Maya appeared, dropping into the chair across from her. "You look thoughtful."

"I keep thinking about what's next." Alex turned her cup in her hands. "This was one problem. One reactor. There are thousands of problems like this—hidden patterns, unexplained failures, data no one's asking the right questions about."

"So you want to keep doing this?"

"I want to get better at it." Alex thought about the semester—the struggles, the breakthroughs, the late nights and unexpected connections. "I came back to school because I wanted to understand, not just apply. I think I understand a little more now."

Jordan and Sam joined them, the team reuniting one last time.

"Frank Morrison called me," Sam said. "He wants to know if we can consult on another plant."

Maya laughed. "Didn't he hate us at first?"

"He hated what he didn't understand." Alex smiled. "Once he understood, he became an advocate. That's usually how it works."

She looked around at her team—the CS expert who learned chemistry, the pharma veteran who found his voice, the star student who discovered humility. They'd started as strangers. They'd become colleagues. Maybe friends.

"Whatever's next," Alex said, "we should do it together."

The Data Academy had one final update to her profile: *Rank: Data Scientist*

But that wasn't the real accomplishment. The real accomplishment was simpler: she'd walked into a room full of skeptics, with data and models and uncertainty, and she'd helped them see what they'd been missing.

That was the work. And it was only beginning.

---

*End of The Catalyst Crisis*

---

**Congratulations on completing Data Science and Machine Learning in Chemical Engineering!** You've learned the same tools Alex and her team used to solve the Catalyst Crisis. Now it's your turn to find the patterns hiding in your own data.